In [61]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense

# Set random seed for reproducibility
np.random.seed(21)
random.seed(21)
tf.random.set_seed(21)

In [62]:
# Read in the independent and target variable datasets

stock = 'AAPL'
window_size = 20
pred_horizon = 10

X = pd.read_parquet(stock + '_X_' + str(window_size) + 'D.gzip')
y = pd.read_parquet(stock + '_y_' + str(pred_horizon) + 'D.gzip')
y = y.cumsum(axis=1)
y = y.iloc[:, -1:]
y_binary = (y > 0).astype(int)

In [63]:
# Split the dataset into training, validation and test datasets

num_features = 20

q_80 = int(len(X) * .8)
q_90 = int(len(X) * .9)

X_train, y_train = X[:q_80].to_numpy(), y_binary[:q_80].to_numpy()
X_val, y_val = X[q_80:q_90].to_numpy(), y_binary[q_80:q_90].to_numpy()
X_test, y_test = X[q_90:].to_numpy(), y_binary[q_90:].to_numpy()

X_train = X_train.reshape((-1, window_size, num_features))
X_val = X_val.reshape((-1, window_size, num_features))
X_test = X_test.reshape((-1, window_size, num_features))

In [64]:
# Define the early stopping callback to be used in all neural networks

early_stop = EarlyStopping(
    monitor='val_accuracy',        
    patience=5,                # wait 5 epochs for improvement
    restore_best_weights=True  # roll back to best weights
)

# Calculate class weights from training data

y_train_1D = y_train.reshape(-1)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_1D),
    y=y_train_1D
)
class_weights = dict(enumerate(class_weights))

In [65]:
# Fit an RNN model to the dataset

model_RNN = Sequential([
    SimpleRNN(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_RNN.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_RNN.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


209/209 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4861 - loss: 0.6944 - val_accuracy: 0.4826 - val_loss: 0.6994
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5277 - loss: 0.6929 - val_accuracy: 0.4850 - val_loss: 0.7012
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5292 - loss: 0.6927 - val_accuracy: 0.4946 - val_loss: 0.6997
Epoch 4/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5332 - loss: 0.6919 - val_accuracy: 0.5306 - val_loss: 0.6934
Epoch 5/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5333 - loss: 0.6910 - val_accuracy: 0.5462 - val_loss: 0.6897
Epoch 6/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5296 - loss: 0.6904 - val_accuracy: 0.5474 - val_loss: 0.6907
Epoch 7/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5341 - loss: 0.6895 - val_accuracy: 0.5462 - val_loss: 0.6914
Epoch 8/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5389 - loss: 0.6893 - val_accuracy: 0.549

In [66]:
# Fit an LSTM model to the dataset

model_LSTM = Sequential([
    LSTM(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_LSTM.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_LSTM.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

Epoch 1/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5143 - loss: 0.6934 - val_accuracy: 0.5066 - val_loss: 0.7099
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5248 - loss: 0.6935 - val_accuracy: 0.5090 - val_loss: 0.7102
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5223 - loss: 0.6930 - val_accuracy: 0.5138 - val_loss: 0.7096
Epoch 4/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5258 - loss: 0.6920 - val_accuracy: 0.5138 - val_loss: 0.7094
Epoch 5/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5290 - loss: 0.6909 - val_accuracy: 0.5138 - val_loss: 0.7092
Epoch 6/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5317 - loss: 0.6892 - val_accuracy: 0.5150 - val_loss: 0.7078
Epoch 7/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5392 - loss: 0.6870 - val_accuracy: 0.5282 - val_loss: 0.7051
Epoch 8/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5448 - loss: 0.6841 - 

In [67]:
# Fit a GRU model to the dataset

model_GRU = Sequential([
    GRU(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_GRU.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_GRU.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

Epoch 1/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.4739 - loss: 0.6985 - val_accuracy: 0.4886 - val_loss: 0.7015
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5178 - loss: 0.6936 - val_accuracy: 0.5018 - val_loss: 0.7038
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5233 - loss: 0.6931 - val_accuracy: 0.5030 - val_loss: 0.7063
Epoch 4/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5239 - loss: 0.6927 - val_accuracy: 0.5090 - val_loss: 0.7083
Epoch 5/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5230 - loss: 0.6924 - val_accuracy: 0.5090 - val_loss: 0.7081
Epoch 6/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5268 - loss: 0.6919 - val_accuracy: 0.5114 - val_loss: 0.7079
Epoch 7/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5295 - loss: 0.6913 - val_accuracy: 0.5198 - val_loss: 0.7084
Epoch 8/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5296 - loss: 0.6907 - 

In [73]:
def evaluate_classifier(y_true, y_probs, model_name='Model', threshold=0.5):

    # Generate binary predictions based on threshold
    y_pred = (y_probs > threshold).astype(int)

    # Classification report
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    # AUC
    auc = roc_auc_score(y_true, y_probs)

    # Build a summary row
    metrics = {
        "Model": model_name,
        "Accuracy": report["accuracy"],
        "Precision (0)": report["0"]["precision"],
        "Recall (0)": report["0"]["recall"],
        "F1 (0)": report["0"]["f1-score"],
        "Precision (1)": report["1"]["precision"],
        "Recall (1)": report["1"]["recall"],
        "F1 (1)": report["1"]["f1-score"],
        "AUC": auc
    }

    df_metrics = pd.DataFrame([metrics])
    df_metrics = df_metrics.apply(lambda x: round(x, 4))
    return df_metrics

In [74]:
# Predict the probabilities for the test dataset using the RNN model

y_RNN = model_RNN.predict(X_test)

fpr_RNN, tpr_RNN, thresholds_RNN = roc_curve(y_test, y_RNN)

# Choose threshold that gives better trade-off
J_RNN = tpr_RNN - fpr_RNN
optimal_idx_RNN = np.argmax(J_RNN)
optimal_threshold_RNN = thresholds_RNN[optimal_idx_RNN]

# Recalculate predictions
y_RNN_opt = (y_RNN > optimal_threshold_RNN).astype(int)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [75]:
# Predict the probabilities for the test dataset using the LSTM model

y_LSTM = model_LSTM.predict(X_test)

fpr_LSTM, tpr_LSTM, thresholds_LSTM = roc_curve(y_test, y_LSTM)

# Choose threshold that gives better trade-off
J_LSTM = tpr_LSTM - fpr_LSTM
optimal_idx_LSTM = np.argmax(J_LSTM)
optimal_threshold_LSTM = thresholds_LSTM[optimal_idx_LSTM]

# Recalculate predictions
y_LSTM_opt = (y_LSTM > optimal_threshold_LSTM).astype(int)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [76]:
# Predict the probabilities for the test dataset using the GRU model

y_GRU = model_GRU.predict(X_test)

fpr_GRU, tpr_GRU, thresholds_GRU = roc_curve(y_test, y_GRU)

# Choose threshold that gives better trade-off
J_GRU = tpr_GRU - fpr_GRU
optimal_idx_GRU = np.argmax(J_GRU)
optimal_threshold_GRU = thresholds_GRU[optimal_idx_GRU]

# Recalculate predictions
y_GRU_opt = (y_GRU > optimal_threshold_GRU).astype(int)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [77]:
results_RNN = evaluate_classifier(y_test, y_RNN_opt, model_name='RNN', threshold=optimal_threshold_RNN)
results_LSTM = evaluate_classifier(y_test, y_LSTM_opt, model_name='LSTM', threshold=optimal_threshold_LSTM)
results_GRU = evaluate_classifier(y_test, y_GRU_opt, model_name='GRU', threshold=optimal_threshold_GRU)

all_results = pd.concat([results_RNN, results_LSTM, results_GRU])
display(all_results)


,Model,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),AUC
0,RNN,0.5635,0.469,0.5828,0.5198,0.6594,0.5504,0.6000,0.5666
0,LSTM,0.6019,0.800,0.0237,0.0460,0.5995,0.9960,0.7485,0.5098
0,GRU,0.5743,0.483,0.7130,0.5759,0.7104,0.4798,0.5728,0.5964
